In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score, StratifiedKFold


# 1️⃣ Đọc dữ liệu
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dataset.csv')
print("Dataset shape:", data.shape)

# 2️⃣ Xử lý dữ liệu
target_col = data.columns[0]
X = data.iloc[:, 1:]  # Các đặc trưng
y = data[target_col]  # Nhãn

# Nếu y là số liên tục, chuyển thành nhãn phân loại
if np.issubdtype(y.dtype, np.number) and y.nunique() > 10:
    threshold = y.median()
    y = np.where(y < threshold, 0, 1)

# Chia tập dữ liệu
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# 3️⃣ Tối ưu Random Forest
rf = RandomForestClassifier(
    n_estimators=1000,         # Tăng số lượng cây
    max_depth=100,             # Tăng độ sâu tối đa
    min_samples_split=2,       # Giảm min split để cây học sâu hơn
    min_samples_leaf=1,        # Cho phép các node nhỏ nhất
    max_features='log2',       # Dùng nhiều đặc trưng hơn khi chia nhánh
    bootstrap=False,           # Không dùng bootstrap để lấy toàn bộ dữ liệu
    class_weight='balanced',   # Giải quyết mất cân bằng dữ liệu
    random_state=42,
    n_jobs=-1                  # Dùng tất cả CPU để train nhanh hơn
)
# 4️⃣ Cross-Validation trước khi train
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(rf, X_train, y_train, cv=cv, scoring='accuracy', n_jobs=-1)

# In kết quả Cross-Validation
print(f"📊 Mean CV Accuracy: {np.mean(cv_scores) * 100:.2f}%")
print(f"📊 CV Accuracy Scores: {cv_scores * 100}")

# 5️⃣ Train mô hình
rf.fit(X_train, y_train)

# 6️⃣ Dự đoán & Đánh giá
y_pred = rf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("🔥 Random Forest Accuracy (Final Test): {:.2f}%".format(acc * 100))
print("\n📌 Classification Report:\n", classification_report(y_test, y_pred))
print("\n📌 Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Dataset shape: (247950, 42)
📊 Mean CV Accuracy: 96.35%
📊 CV Accuracy Scores: [96.22459725 96.46210033 96.26940916 96.35231117 96.45985974]
🔥 Random Forest Accuracy (Final Test): 97.16%

📌 Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.98      0.97     12821
           1       0.98      0.96      0.97     11974

    accuracy                           0.97     24795
   macro avg       0.97      0.97      0.97     24795
weighted avg       0.97      0.97      0.97     24795


📌 Confusion Matrix:
 [[12565   256]
 [  449 11525]]
